# Imports & Settings


In [1]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan
import wandb
import random
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier
# utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [2]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})
test_prep = pd.read_csv('data/test_prepared.csv', index_col='id', dtype={'type': str})
train_prep = pd.read_csv('data/train_prepared.csv', index_col='id', dtype={'type': str})
low = pd.read_csv('data/prep_low_thres.csv', index_col='id', dtype={'type': str})
high = pd.read_csv('data/prep_high_thres.csv', index_col='id', dtype={'type': str})

# setup data for classifier

In [3]:
data = low.copy()
data = data[data.source=='train']
data = data.drop(columns=['source'])

In [6]:
data = train_prep.copy()

features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

# setup rf classifier

In [7]:
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)
probs = rfc.predict_proba(X_test)
val_acc = accuracy_score(y_test, y_pred)

y_pred = rfc.predict(X_train)
probs = rfc.predict_proba(X_train)
train_acc = accuracy_score(y_train, y_pred)

print(train_acc, val_acc)
#0.985204081632653 0.8952380952380953
#0.991938775510204 0.8940476190476191
#0.9860204081632653 0.8942857142857142

0.9860204081632653 0.8942857142857142


# sweeping

In [6]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan
import wandb
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

# Define sweep config
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': 
    {
        'split': {'values': [2, 5, 10, 100]},
        'depth': {'values': [3, 6, 10, 50, 100, 1000]},
        'leaf': {'values': [2, 6, 10, 50, 100, 1000, 5000]},
        'estimators': {'values': [200, 500, 1000, 2000]},
        'features': {'values': [None, 'sqrt', 'log2']},

     }
}

project = 'rf'
# Initialize sweep by passing in config. (Optional) Provide a name of the project.
sweep_id = wandb.sweep(sweep=sweep_configuration, project=project)

def main():
    run = wandb.init(project=project)

    # note that we define values from `wandb.config` instead 
    # of defining hard values 
    split = wandb.config.split
    depth = wandb.config.depth
    leaf = wandb.config.leaf
    estimators = wandb.config.estimators
    feat = wandb.config.features
    

    # -------------------------- usual training code starts here  -------------------------------------
    
    rfc = RandomForestClassifier(n_estimators=estimators, max_depth=depth, min_samples_leaf=leaf, max_features=feat, min_samples_split=split, random_state=42)
    rfc.fit(X_train, y_train)

    y_pred = rfc.predict(X_test)
    val_acc = accuracy_score(y_test, y_pred)
    
    y_pred = rfc.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred)

    print(train_acc, val_acc)

    # -------------------------- ends here  -------------------------------------
    

    wandb.log({
      'train_acc': train_acc,
      'val_acc': val_acc,
    })

# Start sweep job.
wandb.agent(sweep_id, function=main)

Create sweep with ID: etjoorck
Sweep URL: https://wandb.ai/museum-item-classification/rf/sweeps/etjoorck


wandb: Agent Starting Run: 3je55mvn with config:
wandb: 	depth: 1000
wandb: 	estimators: 2000
wandb: 	features: sqrt
wandb: 	leaf: 1000
wandb: 	split: 5
wandb: Currently logged in as: tillwenke (museum-item-classification). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


0.27346938775510204 0.254047619047619


train_acc,▁
val_acc,▁
train_acc,0.27347
val_acc,0.25405


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: gv0lnje7 with config:
wandb: 	depth: 1000
wandb: 	estimators: 200
wandb: 	features: None
wandb: 	leaf: 2
wandb: 	split: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: bdt73kgj with config:
wandb: 	depth: 1000
wandb: 	estimators: 1000
wandb: 	features: None
wandb: 	leaf: 2
wandb: 	split: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


0.9430612244897959 0.893095238095238


train_acc,▁
val_acc,▁
train_acc,0.94306
val_acc,0.8931


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: htel75d3 with config:
wandb: 	depth: 1000
wandb: 	estimators: 2000
wandb: 	features: None
wandb: 	leaf: 2
wandb: 	split: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Ctrl + C detected. Stopping sweep.


# save model

In [14]:
import pickle
pickle.dump(rfc, open('./models/rf/first_try', 'wb'))
loaded_model = pickle.load(open('./models/rf/first_try', 'rb'))

# submission from model

In [ ]:
test_set = test_prep.drop('type', axis=1)

In [ ]:
submission = pd.DataFrame({'id': test_set.index ,'type': bst.predict(test_set)})

In [ ]:
type_lookup = pd.read_csv('data/type_lookup.csv')

In [ ]:
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())

In [ ]:
submission.to_csv('submissions/some_rf_model.csv', index=False)